In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

  HTTP error 403 while getting http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
  Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import copy
import re
import numpy as np
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import codecs
import random
import csv

from tqdm import tqdm 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from torch.autograd import Variable
from sklearn.metrics import f1_score

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
data_directory = "/content/drive/My Drive/OffensEval/data/startkit/training-v1/offenseval-training-v1.tsv"
print("Reading offenseval-training-v1.tsv...")
training_data = pd.read_csv(data_directory, sep='\t', header=0)
tweets = training_data[["tweet"]]
task_a_labels = training_data[["subtask_a"]]
task_b_labels = training_data.query("subtask_a == 'OFF'")[["subtask_b"]]
task_c_labels = training_data.query("subtask_b == 'TIN'")[["subtask_c"]]
clean_tweets = copy.deepcopy(tweets)

Reading offenseval-training-v1.tsv...


In [0]:
def clean_data(tweet):
  noises = ['@USER', 'URL', '\'s', '\'ve', 'n\'t', '\'re', '\'ll', '\'d']
  
  for noise in noises:
    tweet = tweet.replace(noise, '')
  
  return re.sub(r'[^a-zA-Z]', ' ', tweet)


def tokenize(tweet):
  return word_tokenize(tweet.lower())


def remove_stop_words(tweets):
  clean_tweets = []
  stop_words = set(stopwords.words('english'))
  
  for tweet in tweets:
    if tweet not in stop_words:
      if tweet.replace(' ', '') != '':
        if len(tweet) > 1:
          clean_tweets.append(tweet)
  
  return clean_tweets


def lemmatize_and_stem(tweets):
  clean_tweets = []
  lemmatizer = WordNetLemmatizer()
  stemmer = PorterStemmer()
  
  for tweet in tweets:
    tweet = lemmatizer.lemmatize(tweet)
    tweet = stemmer.stem(tweet)
    if len(tweet) > 1:
      clean_tweets.append(tweet)
  
  return clean_tweets


def word_to_index(tweets):
  vocabulary = []
  
  for tweet in tweets:
    for token in tweet:
      if token not in vocabulary:
        vocabulary.append(token)
  
  word2index = {w: idx for (idx, w) in enumerate(vocabulary)}
  
  return word2index


def label_to_index(labels):
  dictionary = []
  
  for label in labels:
    if label not in dictionary:
      dictionary.append(label)
  
  label2index = {l: idx for (idx, l) in enumerate(dictionary)}
  
  return label2index


def tweet_to_tensor(tweets, word2index, max_len):
  vectorized_tweets = [[word2index[token] for token in tweet if token in word2index] for tweet in tweets]
  tweet_tensor = Variable(torch.zeros((len(vectorized_tweets), max_len))).long()
  tweet_lengths = [len(tweet) for tweet in vectorized_tweets]
  
  for index, (tweet, tweetlen) in enumerate(zip(vectorized_tweets, tweet_lengths)):
    tweet_tensor[index, :tweetlen] = torch.LongTensor(tweet)
  
  return tweet_tensor


def get_tensors_by_label(tensors, labels, keyword, max_len):
  if tensors.shape[0] != len(labels):
    print("Unmatching sizes")
    return
  
  length = labels.count(keyword)
  tweet_tensor = Variable(torch.zeros((length, max_len))).long()
  index = 0
  
  for tensor, label in zip(tensors, labels):
    if label == keyword:
      tweet_tensor[index] = tensor
      index += 1
  
  return tweet_tensor


def label_to_tensor(labels, label2index):
  vectorized_labels = [label2index[label] for label in labels if label in label2index]
  label_tensor = torch.LongTensor(vectorized_labels)
  
  return label_tensor

In [0]:
tqdm.pandas(desc="Cleaning Data...")
clean_tweets['tweet'] = tweets['tweet'].progress_apply(clean_data)
tqdm.pandas(desc="Tokenizing Data...")
clean_tweets['tokens'] = clean_tweets['tweet'].progress_apply(tokenize)
tqdm.pandas(desc="Removing Stop Words...")
clean_tweets['tokens'] = clean_tweets['tokens'].progress_apply(remove_stop_words)
tqdm.pandas(desc="Lemmatizing And Stemming...")
clean_tweets['tokens'] = clean_tweets['tokens'].progress_apply(lemmatize_and_stem)

Lemmatizing And Stemming...: 100%|██████████| 13240/13240 [00:06<00:00, 1931.46it/s]


In [0]:
tokenized_tweets = clean_tweets['tokens'].tolist()

tweet_lengths = [len(tweet) for tweet in tokenized_tweets]
max_length = np.max(np.array(tweet_lengths))

word2index = word_to_index(tokenized_tweets)

labels_a = task_a_labels['subtask_a'].values.tolist()
labels_b = task_b_labels['subtask_b'].values.tolist()
labels_c = task_c_labels['subtask_c'].values.tolist()

a2index = label_to_index(labels_a)
b2index = label_to_index(labels_b)
c2index = label_to_index(labels_c)

tweets_a_tensor = tweet_to_tensor(tokenized_tweets, word2index, max_length)
tweets_b_tensor = get_tensors_by_label(tweets_a_tensor, labels_a, "OFF", max_length)
tweets_c_tensor = get_tensors_by_label(tweets_b_tensor, labels_b, "TIN", max_length)

labels_a_tensor = label_to_tensor(labels_a, a2index)
labels_b_tensor = label_to_tensor(labels_b, b2index)
labels_c_tensor = label_to_tensor(labels_c, c2index)

print("Task A tensor size:")
print(tweets_a_tensor.shape)
print("Task B tensor size:")
print(tweets_b_tensor.shape)
print("Task C tensor size:")
print(tweets_c_tensor.shape)

Task A tensor size:
torch.Size([13240, 39])
Task B tensor size:
torch.Size([4400, 39])
Task C tensor size:
torch.Size([3876, 39])


In [0]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
  print('cuda')
  device = torch.device('cuda')
else:
  print('cpu')
  device = torch.device('cpu')

cuda


In [0]:
def train_valid_split(dataset, labels, validation_split):
  dataset_size = dataset.shape[0]
  
  indices = np.arange(dataset_size)
  np.random.shuffle(indices)
  dataset = dataset[indices]
  labels = labels[indices]
  
  split = int(np.floor(validation_split * dataset_size))
  train_data = dataset[split:]
  valid_data = dataset[:split]
  train_labels = labels[split:]
  valid_labels = labels[:split]
  
  return train_data, valid_data, train_labels, valid_labels

In [0]:
class LSTMClassifier(nn.Module):
  
  def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, dropout):
    super(LSTMClassifier, self).__init__()
    self.hidden_dim = hidden_dim
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)
    self.hidden2label = nn.Linear(hidden_dim, label_size)
    self.dropout = nn.Dropout(dropout)
  
  def init_hidden(self, batch_size):
    h0 = Variable(torch.zeros(1, batch_size, self.hidden_dim))
    c0 = Variable(torch.zeros(1, batch_size, self.hidden_dim))
    return (h0, c0)
  
  def forward(self, x, batch_size):
    self.hidden = self.init_hidden(batch_size)
    embedded = self.embedding(x)
    embedded = embedded.view(x.shape[1], batch_size, -1)
    lstm_out, self.hidden = self.lstm(embedded, self.hidden)
    dropout = self.dropout(lstm_out)
    preds = self.hidden2label(dropout[-1])
    return preds

In [0]:
def get_accuracy(output, target):
  num_correct = 0.0
  for i, o in enumerate(output):
    probs = torch.softmax(o, 0)
    _, pred_label = torch.max(probs, 0)
    if pred_label == target[i]:
      num_correct += 1
  acc = num_correct / len(target)
  return acc


def get_f1_score(output, target):
  preds = []
  for o in output:
    probs = torch.softmax(o, 0)
    _, pred_label = torch.max(probs, 0)
    preds.append(pred_label)
  score = f1_score(preds, target, average='macro')
  return score

In [0]:
def train_epoch(model, data, labels, optimizer, loss_fn):
  model.train()
  optimizer.zero_grad()
  
  preds = model(data, len(data))
  loss = loss_fn(preds, labels)
  acc = get_accuracy(preds, labels)
    
  loss.backward()
  optimizer.step()
  
  train_loss = loss.item()
  
  return acc, train_loss


def evaluate(model, data, labels, loss_fn):
  model.eval()
  
  with torch.no_grad():
    preds = model(data, len(data))
    loss = loss_fn(preds, labels)
    acc = get_accuracy(preds, labels)
    valid_loss = loss.item()
  
  return acc, valid_loss


def train(model, data, labels, optimizer, epochs=1, validation_split=0.2, print_every=5):
  class_count = np.array([len(np.where(labels == t)[0]) for t in np.unique(labels)])
  weight = torch.FloatTensor(class_count/len(labels))
  loss_fn = nn.CrossEntropyLoss(weight=weight)
  
  if USE_GPU:
    model = model.to(device=device)
    data = data.to(device=device)
    labels = labels.to(device=device)
  
  for e in range(1, epochs+1):
    train_data, valid_data, train_labels, valid_labels = train_valid_split(data, labels, validation_split)
    
    train_acc, train_loss = train_epoch(model, train_data, train_labels, optimizer, loss_fn)
    valid_acc, valid_loss = evaluate(model, valid_data, valid_labels, loss_fn)
    
    if e % print_every == 0:
      print(f'| Epoch: {e:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')
    
  return valid_acc

In [0]:
!pip install GPy GPyOpt

import GPy
import GPyOpt
from GPyOpt.methods import BayesianOptimization

In [0]:
# Use GPyOpt for hyper-parameter search
domain = [{'name': 'embedding_dim', 'type': 'discrete', 'domain': (100, 120, 140, 160, 180, 200)},
          {'name': 'hidden_dim', 'type': 'discrete', 'domain': (32, 64, 96, 128, 160, 192)},
          {'name': 'drop_out', 'type': 'continuous', 'domain': (0.1, 0.5)},
          {'name': 'lr', 'type': 'continuous', 'domain': (0.0001, 0.001)},
          {'name': 'weight_decay', 'type': 'continuous', 'domain': (0, 0.5)}
         ]

In [0]:
INPUT_DIM = len(word2index)
LABEL_SIZE = 2

def taskA_tuning(params):
  param = params[0]
  # LSTMClassifier(vocab_size, embedding_dim, hidden_dim, label_size, dropout)
  model = LSTMClassifier(INPUT_DIM, int(param[0]), int(param[1]), 2, param[2])
  optimizer = optim.Adam(model.parameters(), lr=param[3], weight_decay=param[4])
  acc = train(model, tweets_a_tensor, labels_a_tensor, optimizer, epochs=10, print_every=100)
  return acc

taskA_opt = BayesianOptimization(f=taskA_tuning,
                                 domain=domain,
                                 model_type='GP',
                                 acquisition_type='EI',
                                 acquisition_jitter=0.05,
                                 maximize=True)

taskA_opt.run_optimization(max_iter=50)

In [0]:
print("The hyperparameters used for the model for Sub-Task A are")
print(f'embedding_dim: {int(taskA_opt.X[-1][0])}')
print(f'hidden_dim: {int(taskA_opt.X[-1][1])}')
print(f'drop_out: {taskA_opt.X[-1][2]:.2f}')
print(f'learning_rate: {taskA_opt.X[-1][3]:.4f}')
print(f'weight_decay: {taskA_opt.X[-1][4]:.4f}')

The hyperparameters used for the model for Sub-Task A are
embedding_dim: 100
hidden_dim: 128
drop_out: 0.11
learning_rate: 0.0006
weight_decay: 0.1309


In [0]:
# LSTMClassifier(vocab_size, embedding_dim, hidden_dim, label_size, dropout)
test_model_A = LSTMClassifier(INPUT_DIM, int(taskA_opt.X[-1][0]), int(taskA_opt.X[-1][1]), LABEL_SIZE, taskA_opt.X[-1][2])
optimizer_A = optim.Adam(test_model_A.parameters(), lr=taskA_opt.X[-1][3], weight_decay=taskA_opt.X[-1][4])
acc_A = train(test_model_A, tweets_a_tensor, labels_a_tensor, optimizer_A, epochs=100)

| Epoch: 05 | Train Loss: 0.591 | Train Acc: 66.71% | Val. Loss: 0.577 | Val. Acc: 67.22% |
| Epoch: 10 | Train Loss: 0.538 | Train Acc: 67.25% | Val. Loss: 0.536 | Val. Acc: 66.46% |
| Epoch: 15 | Train Loss: 0.522 | Train Acc: 67.44% | Val. Loss: 0.525 | Val. Acc: 66.64% |
| Epoch: 20 | Train Loss: 0.509 | Train Acc: 67.79% | Val. Loss: 0.533 | Val. Acc: 65.82% |
| Epoch: 25 | Train Loss: 0.511 | Train Acc: 68.03% | Val. Loss: 0.528 | Val. Acc: 66.21% |
| Epoch: 30 | Train Loss: 0.513 | Train Acc: 67.98% | Val. Loss: 0.520 | Val. Acc: 67.54% |
| Epoch: 35 | Train Loss: 0.511 | Train Acc: 68.37% | Val. Loss: 0.522 | Val. Acc: 66.86% |
| Epoch: 40 | Train Loss: 0.515 | Train Acc: 68.05% | Val. Loss: 0.510 | Val. Acc: 68.66% |
| Epoch: 45 | Train Loss: 0.511 | Train Acc: 68.80% | Val. Loss: 0.527 | Val. Acc: 66.75% |
| Epoch: 50 | Train Loss: 0.515 | Train Acc: 68.23% | Val. Loss: 0.512 | Val. Acc: 69.41% |
| Epoch: 55 | Train Loss: 0.512 | Train Acc: 68.90% | Val. Loss: 0.513 | Val. Ac

In [0]:
_, valid_data, _, valid_labels = train_valid_split(tweets_a_tensor, labels_a_tensor, 0.2)

test_model_A.eval()

with torch.no_grad():
    output = test_model_A(valid_data, len(valid_data))
    acc_A = get_accuracy(output, valid_labels)
    score_A = get_f1_score(output, valid_labels)

print('Validation performance for Sub-Task A')
print(f'Validation Accuracy: {acc_A:.4f}')
print(f'Validation F1 Score: {score_A:.4f}')

Validation performance for Sub-Task A
Validation Accuracy: 0.6962
Validation F1 Score: 0.4198


In [0]:
INPUT_DIM = len(word2index)
LABEL_SIZE = 2

def taskB_tuning(params):
  param = params[0]
  # LSTMClassifier(vocab_size, embedding_dim, hidden_dim, label_size, dropout)
  model = LSTMClassifier(INPUT_DIM, int(param[0]), int(param[1]), LABEL_SIZE, param[2])
  optimizer = optim.SGD(model.parameters(), lr=param[3], weight_decay=param[4])
  acc = train(model, tweets_b_tensor, labels_b_tensor, optimizer, epochs=10, print_every=100)
  return acc

taskB_opt = BayesianOptimization(f=taskB_tuning,
                                 domain=domain,
                                 model_type='GP',
                                 acquisition_type='EI',
                                 acquisition_jitter=0.05,
                                 maximize=True)

taskB_opt.run_optimization(max_iter=50)

In [0]:
print("The hyperparameters used for the model for Sub-Task B are")
print(f'embedding_dim: {int(taskB_opt.X[-1][0])}')
print(f'hidden_dim: {int(taskB_opt.X[-1][1])}')
print(f'drop_out: {taskB_opt.X[-1][2]:.2f}')
print(f'learning_rate: {taskB_opt.X[-1][3]:.4f}')
print(f'weight_decay: {taskB_opt.X[-1][4]:.4f}')

The hyperparameters used for the model for Sub-Task B are
embedding_dim: 140
hidden_dim: 192
drop_out: 0.43
learning_rate: 0.0002
weight_decay: 0.0837


In [0]:
# LSTMClassifier(vocab_size, embedding_dim, hidden_dim, label_size, output_dim, dropout)
test_model_B = LSTMClassifier(INPUT_DIM, int(taskB_opt.X[-1][0]), int(taskB_opt.X[-1][1]), LABEL_SIZE, taskB_opt.X[-1][2])
optimizer_B = optim.Adam(test_model_B.parameters(), lr=taskB_opt.X[-1][3], weight_decay=taskB_opt.X[-1][4])
acc_B = train(test_model_B, tweets_b_tensor, labels_b_tensor, optimizer_B, epochs=100)

| Epoch: 05 | Train Loss: 0.553 | Train Acc: 84.99% | Val. Loss: 0.532 | Val. Acc: 85.40% |
| Epoch: 10 | Train Loss: 0.444 | Train Acc: 87.33% | Val. Loss: 0.412 | Val. Acc: 89.11% |
| Epoch: 15 | Train Loss: 0.373 | Train Acc: 88.38% | Val. Loss: 0.344 | Val. Acc: 87.87% |
| Epoch: 20 | Train Loss: 0.301 | Train Acc: 88.51% | Val. Loss: 0.284 | Val. Acc: 89.32% |
| Epoch: 25 | Train Loss: 0.265 | Train Acc: 88.69% | Val. Loss: 0.250 | Val. Acc: 90.19% |
| Epoch: 30 | Train Loss: 0.242 | Train Acc: 89.35% | Val. Loss: 0.243 | Val. Acc: 88.53% |
| Epoch: 35 | Train Loss: 0.218 | Train Acc: 89.35% | Val. Loss: 0.206 | Val. Acc: 89.18% |
| Epoch: 40 | Train Loss: 0.198 | Train Acc: 89.69% | Val. Loss: 0.194 | Val. Acc: 87.88% |
| Epoch: 45 | Train Loss: 0.192 | Train Acc: 89.37% | Val. Loss: 0.182 | Val. Acc: 90.38% |
| Epoch: 50 | Train Loss: 0.182 | Train Acc: 89.42% | Val. Loss: 0.191 | Val. Acc: 90.59% |
| Epoch: 55 | Train Loss: 0.172 | Train Acc: 89.97% | Val. Loss: 0.173 | Val. Ac

In [0]:
_, valid_data, _, valid_labels = train_valid_split(tweets_b_tensor, labels_b_tensor, 0.2)

test_model_B.eval()

with torch.no_grad():
    output = test_model_B(valid_data, len(valid_data))
    acc_B = get_accuracy(output, valid_labels)
    score_B = get_f1_score(output, valid_labels)

print('Validation performance for Sub-Task B')
print(f'Validation Accuracy: {acc_B:.4f}')
print(f'Validation F1 Score: {score_B:.4f}')

Validation performance for Sub-Task B
Validation Accuracy: 0.9043
Validation F1 Score: 0.4771


In [0]:
INPUT_DIM = len(word2index)
LABEL_SIZE = 3

def taskC_tuning(params):
  param = params[0]
  # LSTMClassifier(vocab_size, embedding_dim, hidden_dim, label_size, dropout)
  model = LSTMClassifier(INPUT_DIM, int(param[0]), int(param[1]), LABEL_SIZE, param[2])
  optimizer = optim.SGD(model.parameters(), lr=param[3], weight_decay=param[4])
  acc = train(model, tweets_c_tensor, labels_c_tensor, optimizer, epochs=10, print_every=100)
  return acc

taskC_opt = BayesianOptimization(f=taskC_tuning,
                                 domain=domain,
                                 model_type='GP',
                                 acquisition_type='EI',
                                 acquisition_jitter=0.05,
                                 maximize=True)

taskC_opt.run_optimization(max_iter=50)

In [0]:
print("The hyperparameters used for the model for Sub-Task C are")
print(f'embedding_dim: {int(taskC_opt.X[-1][0])}')
print(f'hidden_dim: {int(taskC_opt.X[-1][1])}')
print(f'drop_out: {taskC_opt.X[-1][2]:.2f}')
print(f'learning_rate: {taskC_opt.X[-1][3]:.4f}')
print(f'weight_decay: {taskC_opt.X[-1][4]:.4f}')

The hyperparameters used for the model for Sub-Task C are
embedding_dim: 180
hidden_dim: 128
drop_out: 0.36
learning_rate: 0.0002
weight_decay: 0.0170


In [0]:
# LSTMClassifier(vocab_size, embedding_dim, hidden_dim, label_size, dropout)
test_model_C = LSTMClassifier(INPUT_DIM, int(taskC_opt.X[-1][0]), int(taskC_opt.X[-1][1]), LABEL_SIZE, taskC_opt.X[-1][2])
optimizer_C = optim.SGD(test_model_C.parameters(), lr=taskC_opt.X[-1][3], weight_decay=taskC_opt.X[-1][4])
acc_C = train(test_model_C, tweets_c_tensor, labels_c_tensor, optimizer_C, epochs=100)

| Epoch: 05 | Train Loss: 1.096 | Train Acc: 36.85% | Val. Loss: 1.092 | Val. Acc: 30.00% |
| Epoch: 10 | Train Loss: 1.098 | Train Acc: 34.61% | Val. Loss: 1.088 | Val. Acc: 30.79% |
| Epoch: 15 | Train Loss: 1.089 | Train Acc: 37.29% | Val. Loss: 1.082 | Val. Acc: 32.58% |
| Epoch: 20 | Train Loss: 1.082 | Train Acc: 39.06% | Val. Loss: 1.078 | Val. Acc: 33.17% |
| Epoch: 25 | Train Loss: 1.079 | Train Acc: 39.58% | Val. Loss: 1.076 | Val. Acc: 33.90% |
| Epoch: 30 | Train Loss: 1.076 | Train Acc: 39.14% | Val. Loss: 1.067 | Val. Acc: 36.20% |
| Epoch: 35 | Train Loss: 1.071 | Train Acc: 42.79% | Val. Loss: 1.064 | Val. Acc: 45.26% |
| Epoch: 40 | Train Loss: 1.068 | Train Acc: 41.59% | Val. Loss: 1.065 | Val. Acc: 43.15% |
| Epoch: 45 | Train Loss: 1.062 | Train Acc: 43.82% | Val. Loss: 1.057 | Val. Acc: 48.52% |
| Epoch: 50 | Train Loss: 1.055 | Train Acc: 44.57% | Val. Loss: 1.053 | Val. Acc: 50.91% |
| Epoch: 55 | Train Loss: 1.050 | Train Acc: 46.26% | Val. Loss: 1.047 | Val. Ac

In [0]:
_, valid_data, _, valid_labels = train_valid_split(tweets_c_tensor, labels_c_tensor, 0.2)

test_model_C.eval()

with torch.no_grad():
    output = test_model_C(valid_data, len(valid_data))
    acc_C = get_accuracy(output, valid_labels)
    score_C = get_f1_score(output, valid_labels)

print('Validation performance for Sub-Task C')
print(f'Validation Accuracy: {acc_C:.4f}')
print(f'Validation F1 Score: {score_C:.4f}')

Validation performance for Sub-Task C
Validation Accuracy: 0.6045
Validation F1 Score: 0.3183


In [0]:
def indices_to_labels(indices, label2index):
  labels = []
  for index in indices:
    for key, num in label2index.items():
      if index == num:
        labels.append(key)
  return labels

In [0]:
testA_dir = "/content/drive/My Drive/OffensEval/data/taskA/testset-taska.tsv"
testB_dir = "/content/drive/My Drive/OffensEval/data/taskB/testset-taskb.tsv"
testC_dir = "/content/drive/My Drive/OffensEval/data/taskC/test_set_taskc.tsv"

testA_data = pd.read_csv(testA_dir, sep='\t', header=0)
testA_ids = testA_data["id"].tolist()
testA_tweets = testA_data[["tweet"]]
clean_testA = copy.deepcopy(testA_tweets)

testB_data = pd.read_csv(testB_dir, sep='\t', header=0)
testB_ids = testB_data["id"].tolist()
testB_tweets = testB_data[["tweet"]]
clean_testB = copy.deepcopy(testB_tweets)

testC_data = pd.read_csv(testC_dir, sep='\t', header=0)
testC_ids = testC_data["id"].tolist()
testC_tweets = testC_data[["tweet"]]
clean_testC = copy.deepcopy(testC_tweets)

tqdm.pandas(desc="Cleaning Data for Task A...")
clean_testA['tweet'] = testA_tweets['tweet'].progress_apply(clean_data)
tqdm.pandas(desc="Tokenizing Data for Task A...")
clean_testA['tokens'] = clean_testA['tweet'].progress_apply(tokenize)
tqdm.pandas(desc="Removing Stop Words for Task A...")
clean_testA['tokens'] = clean_testA['tokens'].progress_apply(remove_stop_words)
tqdm.pandas(desc="Lemmatizing And Stemming for Task A...")
clean_testA['tokens'] = clean_testA['tokens'].progress_apply(lemmatize_and_stem)

tqdm.pandas(desc="Cleaning Data for Task B...")
clean_testB['tweet'] = testB_tweets['tweet'].progress_apply(clean_data)
tqdm.pandas(desc="Tokenizing Data for Task B...")
clean_testB['tokens'] = clean_testB['tweet'].progress_apply(tokenize)
tqdm.pandas(desc="Removing Stop Words for Task B...")
clean_testB['tokens'] = clean_testB['tokens'].progress_apply(remove_stop_words)
tqdm.pandas(desc="Lemmatizing And Stemming for Task B...")
clean_testB['tokens'] = clean_testB['tokens'].progress_apply(lemmatize_and_stem)

tqdm.pandas(desc="Cleaning Data for Task C...")
clean_testC['tweet'] = testC_tweets['tweet'].progress_apply(clean_data)
tqdm.pandas(desc="Tokenizing Data for Task C...")
clean_testC['tokens'] = clean_testC['tweet'].progress_apply(tokenize)
tqdm.pandas(desc="Removing Stop Words for Task C...")
clean_testC['tokens'] = clean_testC['tokens'].progress_apply(remove_stop_words)
tqdm.pandas(desc="Lemmatizing And Stemming for Task C...")
clean_testC['tokens'] = clean_testC['tokens'].progress_apply(lemmatize_and_stem)

tokenized_testA = clean_testA['tokens'].tolist()
tokenized_testB = clean_testB['tokens'].tolist()
tokenized_testC = clean_testC['tokens'].tolist()

testA_tensor = tweet_to_tensor(tokenized_testA, word2index, max_length)
testB_tensor = tweet_to_tensor(tokenized_testB, word2index, max_length)
testC_tensor = tweet_to_tensor(tokenized_testC, word2index, max_length)

print("Test A tensor size:")
print(testA_tensor.shape)
print("Test B tensor size:")
print(testB_tensor.shape)
print("Test C tensor size:")
print(testC_tensor.shape)

Lemmatizing And Stemming for Task C...: 100%|██████████| 213/213 [00:00<00:00, 2193.33it/s]


Test A tensor size:
torch.Size([860, 39])
Test B tensor size:
torch.Size([240, 39])
Test C tensor size:
torch.Size([213, 39])


In [0]:
def get_predictions(output, label2index):
  labels = []
  
  for o in output:
    probs = torch.softmax(o, 0)
    _, pred = torch.max(probs, 0)
    for key, idx in label2index.items():
      if pred == idx:
        labels.append(key)
  
  return labels

In [0]:
test_model_A.eval()

with torch.no_grad():
  output = test_model_A(testA_tensor, len(testA_tensor))

preds_testA = get_predictions(output, a2index)

taskA_df = pd.DataFrame(testA_ids, columns=['id'])
taskA_df['predict'] = preds_testA
taskA_df.to_csv('testA.csv', header=False, index=False)

In [0]:
test_model_B.eval()

with torch.no_grad():
  output = test_model_B(testB_tensor, len(testB_tensor))

preds_testB = get_predictions(output, b2index)

taskB_df = pd.DataFrame(testB_ids, columns=['id'])
taskB_df['predict'] = preds_testB
taskB_df.to_csv('testB.csv', header=False, index=False)

In [0]:
test_model_C.eval()

with torch.no_grad():
  output = test_model_C(testC_tensor, len(testC_tensor))

preds_testC = get_predictions(output, c2index)

taskC_df = pd.DataFrame(testC_ids, columns=['id'])
taskC_df['predict'] = preds_testC
taskC_df.to_csv('testC.csv', header=False, index=False)

NameError: ignored